In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np

class CNN2D(nn.Module):
    def __init__(self):
        super(CNN2D, self).__init__()
        self.conv_layer = nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=(3, 3))
        self.max_pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension (channels=1) for the 2D CNN
        x = self.conv_layer(x)
        x = self.max_pool(x)
        x = self.flatten(x)
        return x


# 1D CNN network for extracting state representative vectors from frequency representative vectors
class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()
        self.conv_layer = nn.Conv1d(in_channels=1, out_channels=num_filters, kernel_size=3)
        self.max_pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension (channels=1) for the 1D CNN
        x = self.conv_layer(x)
        x = self.max_pool(x)
        x = self.flatten(x)
        return x


# LSTM network for state vector prediction
class LSTMNetwork(nn.Module):
    
#     def __init__(self, input_size, hidden_size, num_layers, output_size):
#         super(LSTMNetwork, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x = x.unsqueeze(1)
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         return x
    
    
#     def __init__(self, input_size, hidden_size, num_layers, output_size):
#         super(LSTMNetwork, self).__init__()
#         self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
#         self.output_layer = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x = x.unsqueeze(1)
#         h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
#         out, _ = self.rnn(x, h0)
#         out = self.output_layer(out[:, -1, :])
#         return out
    
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMNetwork, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.output_layer(out[:, -1, :])
        return out
hori = 50
cr = 512
mode = 'lstm'
input_size = 40800 #2400 #4960 #10080 #20320 #40800 
hidden_size = 128
num_layers = 2
output_size = cr*hori 
num_filters = 32
kernel_size = 3
pool_kernel_size = 2
pool_stride = 2


# Load data from CSV
csv_file_path = f'dataset/Uma_{cr}.csv'
df = pd.read_csv(csv_file_path)

input_data = df.values 
target_data = input_data.copy()  
input_windows = []
target_windows = []
window_size = 12

for i in range(len(input_data) - window_size - hori):
    input_windows.append(input_data[i:i+window_size])
    target_windows.append(target_data[i+window_size:i+window_size+hori])
    
print(np.shape(input_windows))
print(np.shape(target_windows))

input_windows = torch.tensor(input_windows, dtype=torch.float)
target_windows = torch.tensor(target_windows, dtype=torch.float)

dataset = TensorDataset(input_windows, target_windows)

train_ratio = 0.7
valid_ratio = 0.2
test_ratio = 1 - train_ratio - valid_ratio
train_size = int(train_ratio * len(input_windows))
valid_size = int((train_ratio + valid_ratio) * len(input_windows)) - int(train_ratio * len(input_windows))
test_size = len(input_windows) - int((train_ratio + valid_ratio) * len(input_windows))

print(train_size, valid_size, test_size)
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_2d_net = CNN2D().to(device)
lstm_net = LSTMNetwork(input_size, hidden_size, num_layers, output_size).to(device)
print(cnn_2d_net)
print(lstm_net)
criterion = nn.MSELoss()
optimizer = optim.Adam(list(cnn_2d_net.parameters()) + list(lstm_net.parameters()), lr=0.001)

num_epochs = 100
avg_loss_lstm = np.zeros(100)
for epoch in range(num_epochs):
    cnn_2d_net.train()
    lstm_net.train()

    total_loss = 0.0
    cnt = 0
    for inp, targets in train_loader:
        if cnt < 198:
#             print(cnt)
            cnt += 1
            inp = inp.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()

            output_2d = cnn_2d_net(inp)
            output_lstm = lstm_net(output_2d)
            output_lstm = torch.reshape(output_lstm, (32,hori,-1))
#             print(output_lstm.size())
            loss = (criterion(output_lstm, targets))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

    avg_loss_lstm[epoch] = np.sqrt(total_loss / len(train_loader))
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss_lstm[epoch]}")

    # Validation loop
    cnn_2d_net.eval()
    lstm_net.eval()

    with torch.no_grad():
        total_val_loss = 0.0
        cnt = 0
        for inp, targets in valid_loader:
            if cnt < 50:
                cnt += 1
                inp = inp.to(device)
                targets = targets.to(device)
#                 print(cnt)
                output_2d = cnn_2d_net(inp)
                output_lstm = lstm_net(output_2d)
                output_lstm = torch.reshape(output_lstm, (32,hori,-1))
                val_loss = (criterion(output_lstm, targets))
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(valid_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss}")

print("Training completed.")





(9938, 12, 512)
(9938, 50, 512)
6956 1988 994
CNN2D(
  (conv_layer): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (max_pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)
LSTMNetwork(
  (lstm): LSTM(40800, 128, num_layers=2, batch_first=True)
  (output_layer): Linear(in_features=128, out_features=25600, bias=True)
)
Epoch 1/100, Train Loss: 53.55538343684056
Epoch 1/100, Validation Loss: 2460.52632843502
Epoch 2/100, Train Loss: 52.80952383118725
Epoch 2/100, Validation Loss: 2412.0263168092756
Epoch 3/100, Train Loss: 52.32511863746046
Epoch 3/100, Validation Loss: 2362.2764756944443
Epoch 4/100, Train Loss: 51.652886216753245
Epoch 4/100, Validation Loss: 2291.383552672371
Epoch 5/100, Train Loss: 50.74738681044379
Epoch 5/100, Validation Loss: 2200.6128937251983
Epoch 6/100, Train Loss: 49.63747063220459
Epoch 6/100, Validation Loss: 2110.5983305431546
Epoch 7/100, Train Loss: 48.6985665694

Epoch 85/100, Train Loss: 27.35354046342597
Epoch 85/100, Validation Loss: 666.3734092106895
Epoch 86/100, Train Loss: 27.196259887772115
Epoch 86/100, Validation Loss: 660.0455254448784
Epoch 87/100, Train Loss: 27.09318664397169
Epoch 87/100, Validation Loss: 652.4657709030878
Epoch 88/100, Train Loss: 26.992786642884425
Epoch 88/100, Validation Loss: 648.299329969618
Epoch 89/100, Train Loss: 26.960703217858605
Epoch 89/100, Validation Loss: 658.8888239784847
Epoch 90/100, Train Loss: 27.07242172146449
Epoch 90/100, Validation Loss: 653.4811401367188
Epoch 91/100, Train Loss: 27.019642639236153
Epoch 91/100, Validation Loss: 653.1562974717882
Epoch 92/100, Train Loss: 26.93435386380343
Epoch 92/100, Validation Loss: 647.5029916914683
Epoch 93/100, Train Loss: 26.855486063901843
Epoch 93/100, Validation Loss: 648.2989317878844
Epoch 94/100, Train Loss: 26.793037354632446
Epoch 94/100, Validation Loss: 647.0781957232763
Epoch 95/100, Train Loss: 27.098966904034985
Epoch 95/100, Valida

In [5]:
# Assuming you already have the test_dataset from the previous code

# Create DataLoader for the test set
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Test loop
cnn_2d_net.eval()
lstm_net.eval()

mse_criterion = nn.MSELoss()
total_mse = 0.0

with torch.no_grad():
    cnt = 0
    for inp, targets in test_loader:
        if cnt < 30:
            cnt += 1
            inp = inp.to(device)
            targets = targets.to(device)

            # Pass data sequentially through the networks
            output_2d = cnn_2d_net(inp)
            output_lstm = lstm_net(output_2d)
            output_lstm = torch.reshape(output_lstm, (32,hori,-1))
#             print(cnt)
            mse_loss = torch.sqrt(mse_criterion(output_lstm, targets))
            total_mse += mse_loss.item()
            print("targets", targets.size())
            print("prediction", output_lstm.size())
            
            target = targets.to('cpu')
            predict = output_lstm.to('cpu')
            print(target.size())
#             torch.save(predict[:,:,:],"predict_rnn_5120.pt")
#             torch.save(target[:,:,:],"target_rnn_5120.pt")
            print(predict.size())
#             print(cnt)
            if cnt == 1:
                print(cnt)
                torch.save(predict[:,:,:],f"predict_{mode}_{cr}_{hori}.pt")
                torch.save(target[:,:,:],f"target_{mode}_{cr}_{hori}.pt")

avg_mse = total_mse / len(test_loader)
print(f"Root Mean Squared Error (RMSE): {avg_mse}")





targets torch.Size([32, 50, 512])
prediction torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
1
targets torch.Size([32, 50, 512])
prediction torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
targets torch.Size([32, 50, 512])
prediction torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
targets torch.Size([32, 50, 512])
prediction torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
targets torch.Size([32, 50, 512])
prediction torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
targets torch.Size([32, 50, 512])
prediction torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
targets torch.Size([32, 50, 512])
prediction torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
targets torch.Size([32, 50, 512])
prediction torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
torch.Size([32, 50, 512])
targets torch.

In [50]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

csv_file_path = 'output/Uma_512/test/predict_512_9.csv'
predict = pd.read_csv(csv_file_path)
# 
csv_file_path = 'output/Uma_512/test/target_512_9.csv'
target = pd.read_csv(csv_file_path)

predict = torch.tensor(predict.values)
target = torch.tensor(target.values)
# target = targets.to('cpu')
# predict = output_lstm.to('cpu')
print(target.size())
print(predict.size())

torch.save(predict[:,:],"predict_stem_512_9.pt")
torch.save(target[:,:],"target_stem_512_9.pt")

torch.Size([979, 512])
torch.Size([979, 512])


In [155]:
import torch
import numpy as np
from scipy.linalg import svd
import matplotlib.pyplot as plt

cr = 128
hori = 3
H_test_stem = torch.load(f'F:\\matrix\\csip\\{cr}\\H_test_{cr}_stem')
H_test_stem = H_test_stem[0:96,:,:]
H_test_lstm = torch.load(f'F:\\matrix\\csip\\{cr}\\H_test_{cr}_lstm')
H_test_rnn = torch.load(f'F:\\matrix\\csip\\{cr}\\H_test_{cr}_rnn')
# H_test_fnn = torch.load(f'F:\\matrix\\{cr}\\H_test_{cr}_fnn_{hori}')

H_hat_stem = torch.load(f'F:\\matrix\\csip\\{cr}\\H_hat_{cr}_stem')
H_hat_stem = H_hat_stem[0:96,:,:]
H_hat_lstm = torch.load(f'F:\\matrix\\csip\\{cr}\\H_hat_{cr}_lstm')
H_hat_rnn = torch.load(f'F:\\matrix\\csip\\{cr}\\H_hat_{cr}_rnn')
# H_hat_fnn = torch.load(f'F:\\matrix\\{cr}\\H_hat_{cr}_fnn_{hori}')


# print(H_test_stem.size())
# print(H_test_lstm.size())
# print(H_hat_stem.size())
# print(H_hat_lstm.size())

def waterfilling(H,P):
    U, g, VT = svd(H)
#     print(np.shape(VT))
    alpha_low = 0 # Initial low
    alpha_high = (P + np.sum(1/g**2)) # Initial high

    stop_threshold = 1e-7 # Stop threshold

    # Iterate while low/high bounds are further than stop_threshold
    while(np.abs(alpha_low - alpha_high) > stop_threshold):
        alpha = (alpha_low + alpha_high) / 2 # Test value in the middle of low/high

        # Solve the power allocation
        p = 1/alpha - 1/g**2 
        p[p < 0] = 0 # Consider only positive power allocation

        # Test sum-power constraints
        if (np.sum(p) > P): # Exceeds power limit => lower the upper bound
            alpha_low = alpha
        else: # Less than power limit => increase the lower bound
            alpha_high = alpha
    return p

def sumrate(ebn,num, H_stem, Ht_stem, H_lstm, Ht_lstm, H_rnn, Ht_rnn, H_fnn, Ht_fnn):


    snr = 10**(ebn/10)
    sr = []
    sr_stem = []
    sr_lstm = []
    sr_rnn  = []
    sr_fnn  = []
    
    power = np.zeros(32)
    p = 0
    capacity = 0
    
    power_stem = np.zeros(32)
    p_stem = 0
    capacity_stem = 0
    
    power_lstm = np.zeros(32)
    p_lstm = 0
    capacity_lstm = 0
    
    power_rnn = np.zeros(32)
    p_rnn = 0
    capacity_rnn = 0
    
    power_fnn = np.zeros(32)
    p_fnn = 0
    capacity_fnn = 0
    
    H_stem = H_stem/np.linalg.norm(H_stem)
    Ht_stem = Ht_stem/np.linalg.norm(Ht_stem)
    
    H_lstm = H_lstm/np.linalg.norm(H_lstm)
    Ht_lstm = Ht_lstm/np.linalg.norm(Ht_lstm)
    
    H_rnn = H_rnn/np.linalg.norm(H_rnn)
    Ht_rnn = Ht_rnn/np.linalg.norm(Ht_rnn)
    
#     H_fnn = H_fnn/np.linalg.norm(H_fnn)
#     Ht_fnn = Ht_fnn/np.linalg.norm(Ht_fnn)
    
#     H = H_test_stem[num]/np.linalg.norm(H_test_stem[num])
#     Ht = H_hat_stem[num]/np.linalg.norm(H_hat_stem[num])
#     print(np.shape(H))
#     p_stem = 0
#     p_stem = waterfilling(H_hat_stem[num],snr)
#     print(p_stem)
    
    for k in range(32):
        power[k] = np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(H_lstm[:,k].conj().T, [32,1]))**2 
        power[k] = power[k]/(np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(H_lstm[:,k].conj().T, [32,1]))**2)
#         power[k] = power[k]/(np.linalg.norm(np.reshape(H_lstm[:,k].conj().T, [32,1])/(np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(H_lstm[:,k].conj().T, [32,1]))**2)))
    
    p = np.abs(power)**2
    for i in range(32):
        capacity += np.log2(1 + p[k]*snr/32)/32;
    sr.append(capacity)
    
    for k in range(32):
        power_stem[k] = np.abs(np.reshape(H_stem[:,k],[1,32]) @ np.reshape(Ht_stem[:,k].conj().T, [32,1]))**2 
        power_stem[k] = power_stem[k]/(np.abs(np.reshape(Ht_stem[:,k],[1,32]) @ np.reshape(Ht_stem[:,k].conj().T, [32,1]))**2)
#         power_stem[k] = power_stem[k]/(np.linalg.norm(np.reshape(Ht_stem[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_stem[:,k],[1,32]) @ np.reshape(Ht_stem[:,k].conj().T, [32,1]))**2)))
    
    p_stem = np.abs(power_stem)**2
    for i in range(32):
        capacity_stem += np.log2(1 + p_stem[k]*snr/32)/32;
    sr_stem.append(capacity_stem)
    
    
    
    for k in range(32):
        power_lstm[k] = np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(Ht_lstm[:,k].conj().T, [32,1]))**2 
        power_lstm[k] = power_lstm[k]/(np.abs(np.reshape(Ht_lstm[:,k],[1,32]) @ np.reshape(Ht_lstm[:,k].conj().T, [32,1]))**2)
#         power_lstm[k] = power_lstm[k]/(np.linalg.norm(np.reshape(Ht_lstm[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_lstm[:,k],[1,32]) @ np.reshape(Ht_lstm[:,k].conj().T, [32,1]))**2)))
    
    p_lstm = np.abs(power_lstm)**2
    for i in range(32):
        capacity_lstm += np.log2(1 + p_lstm[k]*snr/32)/32;
    sr_lstm.append(capacity_lstm)
    
    
    
    for k in range(32):
        power_rnn[k] = np.abs(np.reshape(H_rnn[:,k],[1,32]) @ np.reshape(Ht_rnn[:,k].conj().T, [32,1]))**2 
        power_rnn[k] = power_rnn[k]/(np.abs(np.reshape(Ht_rnn[:,k],[1,32]) @ np.reshape(Ht_rnn[:,k].conj().T, [32,1]))**2)
#         power_rnn[k] = power_rnn[k]/(np.linalg.norm(np.reshape(Ht_rnn[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_rnn[:,k],[1,32]) @ np.reshape(Ht_rnn[:,k].conj().T, [32,1]))**2)))
    
    p_rnn = np.abs(power_rnn)**2
    for i in range(32):
        capacity_rnn += np.log2(1 + p_rnn[k]*snr/32)/32;
    sr_rnn.append(capacity_rnn)
    
    
    
#     for k in range(32):
#         power_fnn[k] = np.abs(np.reshape(H_fnn[:,k],[1,32]) @ np.reshape(Ht_fnn[:,k].conj().T, [32,1]))**2 
#         power_fnn[k] = power_fnn[k]/(np.abs(np.reshape(Ht_fnn[:,k],[1,32]) @ np.reshape(Ht_fnn[:,k].conj().T, [32,1]))**2)
# #         power_fnn[k] = power_fnn[k]/(np.linalg.norm(np.reshape(Ht_fnn[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_fnn[:,k],[1,32]) @ np.reshape(Ht_fnn[:,k].conj().T, [32,1]))**2)))
    
#     p_fnn = np.abs(power_fnn)**2
#     for i in range(32):
#         capacity_fnn += np.log2(1 + p_fnn[k]*snr/32)/32;
#     sr_fnn.append(capacity_fnn)
    
    
    return np.asarray(sr), np.asarray(sr_stem), np.asarray(sr_lstm), np.asarray(sr_rnn), np.asarray(sr_fnn)

    
many = 96
ebn = list(range(0,31,5))
snr = [10**(x/10) for x in ebn]
sr_128 = [[] for i in range(many)]
sr_stem_128 = [[] for i in range(many)]
sr_lstm_128 = [[] for i in range(many)]
sr_rnn_128 = [[] for i in range(many)]
sr_fnn = [[] for i in range(many)]

for i in ebn:
    print(i)
    for j in range(many):
#         print(j)
        sr0, sr1, sr2, sr3, sr4 = sumrate(i,j, H_test_stem[j], H_hat_stem[j], H_test_lstm[j], H_hat_lstm[j], H_test_rnn[j], H_hat_rnn[j], H_test_rnn[j], H_hat_rnn[j])  
        sr_128[j].append(sr0)
        sr_stem_128[j].append(sr1)
        sr_lstm_128[j].append(sr2)
        sr_rnn_128[j].append(sr3)
        sr_fnn[j].append(sr4)




0
5
10
15
20
25
30


In [157]:
%matplotlib qt 

# plt.plot(ebn,np.mean(sr, axis=0), label='perfect, Nt=32, Nc=32', marker = 'X', ms = 7, c = 'm')
plt.plot(ebn,np.mean(sr_stem_512, axis=0), label=r'STEM, $N_{t}=32$, $N_{c}=32$, $\gamma = 1/4$', marker = 'o', ms = 7, c = 'b', linewidth=2)
plt.plot(ebn,np.mean(sr_rnn_512, axis=0), label=r'LSTM, $N_{t}=32$, $N_{c}=32$, $\gamma = 1/4$', marker = 's', ms = 6, c = 'y', linewidth=2)
plt.plot(ebn,np.mean(sr_lstm_512, axis=0), label=r'RNN, $N_{t}=32$, $N_{c}=32$, $\gamma = 1/4$', marker = '*', ms = 8, c = 'r', linewidth=2)

plt.plot(ebn,np.mean(sr_stem_128, axis=0), label=r'STEM, $N_{t}=32$, $N_{c}=32$, $\gamma = 1/8$', marker = 'o', ms = 7, c = 'g', linewidth=2)
plt.plot(ebn,np.mean(sr_rnn_256, axis=0), label=r'LSTM, $N_{t}=32$, $N_{c}=32$, $\gamma = 1/8$', marker = 's', ms = 6, c = 'c', linewidth=2)
plt.plot(ebn,np.mean(sr_lstm_256, axis=0), label=r'RNN, $N_{t}=32$, $N_{c}=32$, $\gamma = 1/8$', marker = '*', ms = 8, c = 'm', linewidth=2)

plt.plot(ebn,np.mean(sr_stem_256, axis=0), label=r'STEM, $N_{t}=32$, $N_{c}=32$, $\gamma = 1/16$', marker = 'o', ms = 7, c = 'k', linewidth=2)
plt.plot(ebn,np.mean(sr_rnn_128, axis=0), label=r'LSTM, $N_{t}=32$, $N_{c}=32$, $\gamma = 1/16$', marker = 's', ms = 6, c = 'tab:brown', linewidth=2)
plt.plot(ebn,np.mean(sr_lstm_128, axis=0), label=r'RNN, $N_{t}=32$, $N_{c}=32$, $\gamma = 1/16$', marker = '*', ms = 8, c = 'tab:orange', linewidth=2)


# plt.plot(ebn,np.mean(sr_fnn , axis=0), label='fnn, Nt=32, Nc=32', marker = '^', ms = 6, c = 'g')
plt.xlabel(r'$E_{b}/N_{0} (dB)$', fontsize = 12)
plt.ylabel("Spectral Efficiency (bps/Hz)", fontsize = 12)
plt.legend()
plt.grid(True)
plt.show()

In [160]:
mse_stem_512 = [-3.61166927774, -3.08451961459, -2.83590100139, -2.51328908166]
mse_stem_256 = [-3.52781830193, -2.96968595073, -2.73166795075, -2.46595246334]
mse_stem_128 = [-3.19103927299, -2.93013932428, -2.61020223448, -2.37514778743]

mse_lstm_128 = [-0.92867873502, -0.90506769294, -0.86569178499, -0.8324122678]
mse_lstm_256 = [-0.80334218188, -0.79374516037, -0.80191702337, -0.7683017809]
mse_lstm_512 = [-0.76180752636, -0.77114846960, -0.78916990878, -0.7550146195]

mse_rnn_128 = [-0.98761271365, -0.95354782156, -0.89697948274, -0.8788428984]
mse_rnn_256 = [-0.89534566440, -0.93959614139, -0.82591949072, -0.8361662438]
mse_rnn_512 = [-0.88676419129, -0.79372990646, -0.80417621209, -0.7994353978]

barWidth = 0.1
fig = plt.subplots(figsize =(12, 8))
horis = [3, 5, 7, 9]

br1 = np.arange(len(horis))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
br5 = [x + barWidth for x in br4]
br6 = [x + barWidth for x in br5]
br7 = [x + barWidth for x in br6]
br8 = [x + barWidth for x in br7]
br9 = [x + barWidth for x in br8]

plt.bar(br1, mse_stem_128, color ='y', width = barWidth,
        edgecolor ='grey', label ='STEM-128')
plt.bar(br2, mse_stem_256, color ='tab:orange', width = barWidth,
        edgecolor ='grey', label ='STEM-256')
plt.bar(br3, mse_stem_512, color ='r', width = barWidth,
        edgecolor ='grey', label ='STEM-512')

plt.bar(br4, mse_rnn_128, color ='mediumseagreen', width = barWidth,
        edgecolor ='grey', label ='LSTM-128')
plt.bar(br5, mse_rnn_256, color ='tab:green', width = barWidth,
        edgecolor ='grey', label ='LSTM-256')
plt.bar(br6, mse_rnn_512, color ='g', width = barWidth,
        edgecolor ='grey', label ='LSTM-512')


plt.bar(br7, mse_lstm_128, color ='xkcd:sky blue', width = barWidth,
        edgecolor ='grey', label ='RNN-128')
plt.bar(br8, mse_lstm_256, color ='tab:blue', width = barWidth,
        edgecolor ='grey', label ='RNN-256')
plt.bar(br9, mse_lstm_512, color ='b', width = barWidth,
        edgecolor ='grey', label ='RNN-512')
 
# Adding Xticks
plt.xlabel('Horizon', fontsize = 12)
plt.ylabel('NMSE (dB) after Channel Reconstruction', fontsize = 12)
plt.xticks([r + 4*barWidth for r in range(len(horis))],
        ['3', '5', '7', '9'])
# plt.ylim(-4, 0)
plt.legend()
plt.show()

In [162]:
rmse_stem_512 = [1.5832, 2.6189, 4.1774, 5.4101]
rmse_stem_256 = [1.9206, 3.0388, 5.2129, 5.6265]
rmse_stem_128 = [2.5506, 3.3334, 4.4479, 6.0265]

rmse_lstm_128 = [27.193055741, 26.6125009050, 26.7770670050, 27.7291775343]
rmse_lstm_256 = [24.729510545, 24.9839926950, 24.7412368050, 25.2967482805]
rmse_lstm_512 = [24.440444529, 23.7080699205, 23.9851315617, 24.3868720531]

rmse_rnn_128 = [25.2789484262, 24.2054104208, 24.9343266487, 24.8131954073]
rmse_rnn_256 = [23.0762034058, 22.9603829382, 23.6214694383, 24.7122474310]
rmse_rnn_512 = [22.4404445290, 22.7080699205, 22.9851315617, 24.3868720531]

barWidth = 0.1
fig = plt.subplots(figsize =(12, 8))
horis = [3, 5, 7, 9]

br1 = np.arange(len(horis))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
br5 = [x + barWidth for x in br4]
br6 = [x + barWidth for x in br5]
br7 = [x + barWidth for x in br6]
br8 = [x + barWidth for x in br7]
br9 = [x + barWidth for x in br8]

plt.bar(br1, rmse_stem_128, color ='y', width = barWidth,
        edgecolor ='grey', label ='STEM-128')
plt.bar(br2, rmse_stem_256, color ='tab:orange', width = barWidth,
        edgecolor ='grey', label ='STEM-256')
plt.bar(br3, rmse_stem_512, color ='r', width = barWidth,
        edgecolor ='grey', label ='STEM-512')

plt.bar(br4, rmse_rnn_128, color ='mediumseagreen', width = barWidth,
        edgecolor ='grey', label ='LSTM-128')
plt.bar(br5, rmse_rnn_256, color ='tab:green', width = barWidth,
        edgecolor ='grey', label ='LSTM-256')
plt.bar(br6, rmse_rnn_512, color ='g', width = barWidth,
        edgecolor ='grey', label ='LSTM-512')


plt.bar(br7, rmse_lstm_128, color ='xkcd:sky blue', width = barWidth,
        edgecolor ='grey', label ='RNN-128')
plt.bar(br8, rmse_lstm_256, color ='tab:blue', width = barWidth,
        edgecolor ='grey', label ='RNN-256')
plt.bar(br9, rmse_lstm_512, color ='b', width = barWidth,
        edgecolor ='grey', label ='RNN-512')
 
# Adding Xticks
plt.xlabel('Horizon', fontsize = 12)
plt.ylabel('RMSE after Compressed Channel Prediction', fontsize = 12)
plt.xticks([r + 4*barWidth for r in range(len(horis))],
        ['3', '5', '7', '9'])
# plt.ylim(-4, 0)
plt.legend()
plt.show()

In [93]:
train_loss_stem = [0.2698, 0.0473, 0.0117, 0.0086, 0.0059, 0.0053, 0.0047, 0.0043, 0.0040, 0.0034, 0.0033, 0.0031, 0.0030, 0.0029, 0.0027, 0.0026, 0.0026, 0.0025, 0.0025, 0.0024, 0.0024, 0.0023, 0.0023, 0.0023, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021]
train_rmse_stem = [0.2983, 0.1806, 0.1396, 0.1140, 0.1230, 0.0893, 0.0834, 0.0800, 0.0763, 0.0759, 0.0712, 0.0690, 0.0672, 0.0654, 0.0651, 0.0634, 0.0634, 0.0621, 0.0612, 0.0614, 0.0605, 0.0600, 0.0597, 0.0593, 0.0591, 0.0590, 0.0586, 0.0585, 0.0584, 0.0581, 0.0580, 0.0579, 0.0579, 0.0577, 0.0576, 0.0575, 0.0575, 0.0575, 0.0574, 0.0573, 0.0573]
# plt.plot(train_rmse_stem)
# plt.plot(avg_loss[0:40])
# plt.xlabel('epochs',fontweight ='bold', fontsize = 12)
# plt.ylabel('RMSE loss during training',fontweight ='bold', fontsize = 12)
# plt.show()


fig = plt.figure()
ax = fig.add_subplot(111)
lns1 = ax.plot(train_rmse_stem, '-', label = 'STEM', color ='y', linewidth=2)
ax2 = ax.twinx()
lns2 = ax2.plot(avg_loss_rnn[0:40], '-', label = 'LSTM', color ='mediumseagreen', linewidth=2)
lns3 = ax2.plot(avg_loss_lstm[0:40], '-', label = 'RNN', color ='xkcd:sky blue', linewidth=2)


ax.grid()
ax.set_xlabel('epochs', fontsize = 12)
ax.set_ylabel('RMSE loss during training for STEM', fontsize = 12)
ax2.set_ylabel('RMSE loss during training for LSTM/RNN', fontsize = 12)
lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc=0)
plt.show()